In [25]:
from tile_classifier import TileClassifier

from keras import backend as K
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

def export_model(saver, model, input_node_names, output_node_name, model_name):
     
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        model_name + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + model_name + '.chkp')

    freeze_graph.freeze_graph('out/' + model_name + '_graph.pbtxt', None, \
        False, 'out/' + model_name + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + model_name + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + model_name + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + model_name + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

In [23]:
if __name__ == '__main__':
    model_path ='tile_classifier.h5'
    data_set_path = './data/data_set_tile'
    model = TileClassifier(data_set_path,model_path)
    model.train_evaluate(epochs=5)
    class_indices = model.get_class_indices()
    print(class_indices)
    

Found 1868 images belonging to 2 classes.
Epoch 1/5
1494/1494 [==============================] - 25s 17ms/step - loss: 0.4925 - acc: 0.7811
Epoch 2/5
1494/1494 [==============================] - 30s 20ms/step - loss: 0.2167 - acc: 0.9096
Epoch 3/5
1494/1494 [==============================] - 32s 21ms/step - loss: 0.0802 - acc: 0.9672
Epoch 4/5
1494/1494 [==============================] - 32s 21ms/step - loss: 0.0317 - acc: 0.9886
Epoch 5/5
374/374 [==============================] - 3s 7ms/step
[0.0020897649770184237, 1.0]
{'non-tiles': 0, 'tiles': 1}


In [ ]:
export_model(tensorflow.train.Saver(), model.classifier, ["conv2d_1_input"], "dense_2/Softmax", 'tile_classifier')